In [1]:
from bs4 import BeautifulSoup as bs
import requests as r
import pandas as pd

In [2]:
# https://www.megamillions.com/Winning-Numbers/Previous-Drawings.aspx
url = 'https://www.megamillions.com/jackpot-history'

In [3]:
def read_html(url, element):
    data = r.get(url)
    html_parse = bs(data.content, 'html.parser')
    return html_parse.find_all(element)


def clean_html(string):
    html_dict = {'\xa0':' ',
                'won by':'-',
                'won in':'-',
                '–':'-'
                }
    
    for key,val in html_dict.items():
        string = string.replace(key,val)
    return(string)


def html_to_df(url):
    df = pd.DataFrame(columns = ['date','amount','winner'])
    index = 0
    winners = read_html(url, 'li')
    for winner in winners:
        win_am = winner.text
        if 'million' in win_am or 'billion' in win_am:
            try:
                df.loc[index] = clean_html(win_am).split(' - ')
            except ValueError:
                df.loc[index] = win_am.replace('$','- ').split(' - ')
            index += 1
    return df


In [4]:
jackpot_winners = html_to_df(url)
jackpot_winners.to_csv('jackpot_winners.csv', index=False)


Winning Numbers

Winning Numbers
Previous Drawings
Check Your Numbers


Winning Numbers
Previous Drawings
Check Your Numbers

How To Play

How To Play
Random Number Generator
Play Responsibly
Difference Between Cash Value and Annuity


How To Play
Random Number Generator
Play Responsibly
Difference Between Cash Value and Annuity

Where To Play


Winners Gallery


About Us

About Us
Jackpot History
Mega Millions Trivia
Resources
Lottery Scams
Security


About Us
Jackpot History
Mega Millions Trivia
Resources
Lottery Scams
Security

FAQS



Media Center
News Releases
Contact Us


Media Center
News Releases
Contact Us






Megaplier 

About Us


Jackpot History


Mega Millions Trivia


Resources


Lottery Scams


Security

January 28, 2022 - $426 million ($292.9 million cash) won in California with a ticket sold in Woodland Hills.
October 22, 2021 - $108 million ($75.2 million cash) won by a family in Lake Havasu City, Ariz.
September 21, 2021 - $431 million ($314.4 million cash) won by

August 28, 2009 - $336 million jackpot - 2 winning tickets ($168 million each):
	
Kevyn Ogawa in San Gabriel, Calif.
James "Jimmy" Groves of the Bronx, N.Y.


Kevyn Ogawa in San Gabriel, Calif.
James "Jimmy" Groves of the Bronx, N.Y.
July 7, 2009 - $133 million won by The Archibald Trust of Queens, N.Y.
May 29, 2009 - $35 million won by the Silverbrush Family of Syracuse, N.Y.
May 15, 2009 - $38 million jackpot - 2 winning tickets ($19 million each):
	
Natalio Perez of Santa Ana, Calif.
Joe and Sue Claffy of Rappahannock County, Va.


Natalio Perez of Santa Ana, Calif.
Joe and Sue Claffy of Rappahannock County, Va.
May 1, 2009 - $227 million jackpot - 3 winning tickets ($75.6 million each):
	
Anneliese Hansen and Brian Cabaud of Mill Valley, Calif.
The Ross County Beneficiary Trust of Chillicothe, Ohio
Joe Denette of Spotsylvania County, Va.


Anneliese Hansen and Brian Cabaud of Mill Valley, Calif.
The Ross County Beneficiary Trust of Chillicothe, Ohio
Joe Denette of Spotsylvania Coun